<a href="https://colab.research.google.com/github/yujinc129-oss/Drama_data_project/blob/main/notebooks/04_encoding_%26_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn. ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer


In [105]:
drama =pd.read_json('https://github.com/yujinc129-oss/Drama_data_project/raw/refs/heads/main/data/processed/03_feature', orient="records")

#인코딩

In [106]:
mlb = MultiLabelBinarizer()

fmt = mlb.fit_transform(drama['genres'])
gen = pd.DataFrame(fmt, columns=mlb.classes_, index=drama.index)


mlb = MultiLabelBinarizer()

fmt = mlb.fit_transform(drama['day'])
day = pd.DataFrame(fmt, columns=mlb.classes_, index=drama.index)

mlb = MultiLabelBinarizer()

fmt = mlb.fit_transform(drama['network'])
net = pd.DataFrame(fmt, columns=mlb.classes_, index=drama.index)


In [107]:
net

,CJ,JPP,JTBC,KBS,MBC,NETFLIX,OCN,SBS,TVN,WEB
0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,1,1,0,0,0,0
4,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2172,0,0,0,1,0,0,0,0,0,0
2173,0,0,0,0,1,0,0,0,0,0
2174,1,0,0,0,0,0,0,0,0,0
2175,0,0,0,0,1,0,0,0,0,0


In [108]:
others = pd.get_dummies(drama[['role','gender']])

In [109]:
X = pd.concat([drama['age'],gen, day, net, others], axis=1)


In [110]:
X

,age,action,comedy,drama,etc,food,hist_war,music,romance,sf,...,MBC,NETFLIX,OCN,SBS,TVN,WEB,role_Main Role,role_Support Role,gender_Female,gender_Male
0,34,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,True,False,False,True
1,34,1,0,1,0,0,1,0,0,0,...,0,1,0,0,0,0,True,False,False,True
2,33,0,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,True,False,False,True
3,32,0,0,1,0,0,0,0,1,0,...,1,1,0,0,0,0,True,False,False,True
4,31,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2172,40,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,False,True,False,True
2173,32,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,True,False,False,True
2174,27,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,True,False,False,True
2175,32,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,True,False,False,True


In [111]:
X_data = X.copy()
data_processed = pd.concat([X_data, drama[['start airing','main_ratio','platform_tier']]], axis=1)

In [112]:
#X변수
X = data_processed
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2177 entries, 0 to 2176
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                2177 non-null   int64  
 1   action             2177 non-null   int64  
 2   comedy             2177 non-null   int64  
 3   drama              2177 non-null   int64  
 4   etc                2177 non-null   int64  
 5   food               2177 non-null   int64  
 6   hist_war           2177 non-null   int64  
 7   music              2177 non-null   int64  
 8   romance            2177 non-null   int64  
 9   sf                 2177 non-null   int64  
 10  society            2177 non-null   int64  
 11  sports             2177 non-null   int64  
 12  thriller           2177 non-null   int64  
 13  youth              2177 non-null   int64  
 14  friday             2177 non-null   int64  
 15  monday             2177 non-null   int64  
 16  saturday           2177 

In [113]:
#y 변수
y= drama['score']

In [114]:
X= X.drop(columns=['gender_Male', 'gender_Female','role_Main Role','role_Support Role','etc','CJ'])

#train/test split

In [115]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)

#Linear Regression

In [116]:
linear = LinearRegression()
linear.fit(X_tr,y_tr)

LinearRegression()

In [117]:
Xpred = linear.predict(X_tr)


In [118]:
cv = cross_validate(linear, X_tr, y_tr, cv=5, n_jobs=1, return_train_score=True)

In [119]:
cv

{'fit_time': array([0.00848866, 0.00570846, 0.00979662, 0.00766587, 0.00607967]),
 'score_time': array([0.0031302 , 0.00294757, 0.0031116 , 0.00429654, 0.00486946]),
 'test_score': array([0.21922456, 0.17910488, 0.17339848, 0.19507878, 0.16782637]),
 'train_score': array([0.21934955, 0.22930089, 0.2295459 , 0.22325372, 0.2265783 ])}

In [120]:
#결과
print(f'TRAIN_SCORE : {cv["train_score"].mean()}  |  TEST_SCORE : {cv["test_score"].mean()}')
print('MAE :',mean_absolute_error(y_tr, Xpred))
print('MSE :',mean_squared_error(y_tr, Xpred))
print('R2 :',r2_score(y_tr, Xpred))

TRAIN_SCORE : 0.22560566805125498  |  TEST_SCORE : 0.186926613640018
MAE : 0.18804047921352715
MSE : 0.06218764899474873
R2 : 0.22193108646303206


#Randomforest

In [121]:
randomf = RandomForestRegressor()

In [122]:
randomf.fit(X_tr, y_tr)
X_pred = randomf.predict(X_tr)

In [123]:
scoring_metrics = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']
cv = cross_validate(randomf, X_tr, y_tr, n_jobs=-1, return_train_score=True, scoring=scoring_metrics )


train_r2_mean = cv['train_r2'].mean()
train_mse_mean = -cv['train_neg_mean_squared_error'].mean() # 음수이므로 -1을 곱해 복원
train_mae_mean = -cv['train_neg_mean_absolute_error'].mean()

test_r2_mean = cv['test_r2'].mean()
test_mse_mean = -cv['test_neg_mean_squared_error'].mean() # 음수이므로 -1을 곱해 복원
test_mae_mean = -cv['test_neg_mean_absolute_error'].mean() # 음수이므로 -1을 곱해 복원
#결과
print(f'TRAIN MAE : {train_mae_mean} / TEST MAE : {test_mae_mean}' )
print(f'TRAIN MSE : {train_mse_mean} / TEST MSE : {test_mse_mean}')
print(f'TRAIN R2 : {train_r2_mean} / TEST R2 : {test_r2_mean}')

TRAIN MAE : 0.0390098320028138 / TEST MAE : 0.1058134382111375
TRAIN MSE : 0.003359251025236315 / TEST MSE : 0.023788527604526842
TRAIN R2 : 0.9579167961787544 / TEST R2 : 0.6998791030606173


##RandomizedSearchCV

In [124]:
from sklearn.model_selection import RandomizedSearchCV
import scipy

In [125]:
# #params = {'max_depth' : scipy.stats.randint(100,150),
#           'min_samples_leaf' : scipy.stats.randint(2,10),
#           }

#randcv = RandomizedSearchCV(randomf, params, n_iter= 100, n_jobs=-1, return_train_score=True)

#randcv.fit(X_tr, y_tr)

#GridSearchCV

In [126]:
from sklearn.model_selection import GridSearchCV

In [127]:
16, 2, 400

(16, 2, 400)

In [128]:
params = { 'n_estimators': [200, 300, 400, 500],  # 이전 최적값으로 고정
    'max_depth': [10, 13, 15], # 이전 최적값(18)보다 의도적으로 낮게 설정
    'min_samples_leaf': [2, 3, 4]                 # 간격을 두고 3개 값 테스트
    #'min_samples_split' : [2, 5, 10],                # 기존 범위에서 일부만 선택
    #'max_features': ['sqrt']
           }

grid = GridSearchCV(randomf, params, n_jobs=-1, cv= 5, return_train_score=True,)
grid.fit(X_tr, y_tr)


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [10, 13, 15],
                         'min_samples_leaf': [2, 3, 4],
                         'n_estimators': [200, 300, 400, 500]},
             return_train_score=True)

In [129]:
grid.best_estimator_.feature_importances_

array([0.08467195, 0.02819487, 0.04735985, 0.05197213, 0.01077283,
       0.03290564, 0.00864807, 0.03108697, 0.04923884, 0.04178524,
       0.00887285, 0.03544064, 0.02465671, 0.02561701, 0.01591986,
       0.01660277, 0.01340681, 0.01328509, 0.0211426 , 0.01383068,
       0.00519094, 0.0186777 , 0.01366966, 0.00967846, 0.03816935,
       0.00868864, 0.01390904, 0.02506993, 0.01045248, 0.15326107,
       0.03954589, 0.08827543])

max_depth=np.int64(21), n_estimators=1000   0.5669713893907954

max_depth=np.int64(19), min_samples_leaf=2, n_estimators=1000  0.5347813665813888

In [130]:
grid.best_score_

np.float64(0.651307074208111)

In [131]:
grid.best_params_

{'max_depth': 15, 'min_samples_leaf': 2, 'n_estimators': 200}

In [132]:
best_model = grid.best_estimator_

# 2. 변수 중요도를 계산합니다.
importances = best_model.feature_importances_

# 3. 컬럼 이름은 학습 데이터(X_tr)에서 바로 가져옵니다.
feature_names = X_tr.columns

# 4. 컬럼 이름과 중요도를 합쳐서 보기 좋게 출력합니다.
feature_importances = pd.Series(importances, index=feature_names).sort_values(ascending=False)

print("--- 변수 중요도 (상위 20개) ---")
print(feature_importances)

--- 변수 중요도 (상위 20개) ---
start airing     0.153261
platform_tier    0.088275
age              0.084672
drama            0.051972
sf               0.049239
comedy           0.047360
society          0.041785
main_ratio       0.039546
NETFLIX          0.038169
thriller         0.035441
hist_war         0.032906
romance          0.031087
action           0.028195
friday           0.025617
TVN              0.025070
youth            0.024657
tuesday          0.021143
JTBC             0.018678
saturday         0.016603
monday           0.015920
SBS              0.013909
wednesday        0.013831
KBS              0.013670
sunday           0.013407
thursday         0.013285
food             0.010773
WEB              0.010452
MBC              0.009678
sports           0.008873
OCN              0.008689
music            0.008648
JPP              0.005191
dtype: float64


In [133]:


results = pd.DataFrame(grid.cv_results_)
results['train-test'] = results['mean_train_score'] - results['mean_test_score']
results_10 = results[['param_max_depth', 'param_min_samples_leaf', 'param_n_estimators',
       'mean_test_score', 'std_test_score','train-test']].sort_values(by=['mean_test_score','train-test'], ascending=[False, True]).head(10)


results_10

,param_max_depth,param_min_samples_leaf,param_n_estimators,mean_test_score,std_test_score,train-test
24,15,2,200,0.651307,0.015239,0.255610
26,15,2,400,0.647707,0.018263,0.259811
27,15,2,500,0.644270,0.014664,0.262856
25,15,2,300,0.643717,0.016733,0.262575
14,13,2,400,0.643486,0.015075,0.252417
12,13,2,200,0.639473,0.018002,0.255299
15,13,2,500,0.638398,0.013760,0.257241
13,13,2,300,0.632413,0.015932,0.261580
2,10,2,400,0.598263,0.015323,0.232758
3,10,2,500,0.597696,0.014346,0.232268


#XGBOOST

In [134]:
from xgboost import XGBRegressor

In [135]:
xg = XGBRegressor()

In [136]:
cv = cross_validate(xg,X_tr, y_tr, n_jobs= -1, return_train_score=True )
print(cv['train_score'].mean())
print(cv['test_score'].mean())

0.9829053087551882
0.7069877138201697


In [137]:
params = {
    'learning_rate': [0.05, 0.06, 0.07],
    'max_depth': [12,13,14],
    'n_estimators': [600],
    'subsample': [0.9],
    'colsample_bytree': [0.5, 0.8, 1.1],

    'gamma': [0.01, 0.02 ],
    'lambda': [0.03, 0.05]
}

grid = GridSearchCV(xg, params, n_jobs=-1, cv= 5, return_train_score=True)
grid.fit(X_tr, y_tr)

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
180 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/xgboost/core.py", line 729, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py", line 1247, in fit
    self._Booster = train(
                    ^^^^^^
  File "/usr/local/lib/python3.11

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.5, 0.8, 1.1],
                         'gamma': [0.01, 0.02], 'lambda': [0.03, 0.05],
                         'learning_rate': [0.05, 0.06, 0.07],
                         'max_depth': [12, 13, 14], 'n_estimators': [600],
                         'subsample': [0.9]},
             return_train_score=True)

In [138]:
grid.best_params_

{'colsample_bytree': 0.8,
 'gamma': 0.01,
 'lambda': 0.05,
 'learning_rate': 0.05,
 'max_depth': 13,
 'n_estimators': 600,
 'subsample': 0.9}

In [139]:
grid.best_score_

np.float64(0.7315383714487034)

In [140]:
results = pd.DataFrame(grid.cv_results_)
results['train-test'] = results['mean_train_score'] - results['mean_test_score']
results_10 = results[['param_colsample_bytree', 'param_learning_rate', 'param_max_depth',
        'param_subsample','mean_train_score', 'mean_test_score','train-test']].sort_values(by=['mean_test_score','train-test'], ascending=[False, True]).head(10)

results_10

,param_colsample_bytree,param_learning_rate,param_max_depth,param_subsample,mean_train_score,mean_test_score,train-test
46,0.8,0.05,13,0.9,0.968723,0.731538,0.237185
37,0.8,0.05,13,0.9,0.969012,0.731292,0.237720
50,0.8,0.06,14,0.9,0.970323,0.730222,0.240101
40,0.8,0.06,13,0.9,0.968793,0.727766,0.241027
36,0.8,0.05,12,0.9,0.967030,0.727287,0.239743
49,0.8,0.06,13,0.9,0.968286,0.726677,0.241609
47,0.8,0.05,14,0.9,0.970147,0.725790,0.244357
41,0.8,0.06,14,0.9,0.970346,0.725651,0.244695
53,0.8,0.07,14,0.9,0.970212,0.725427,0.244785
42,0.8,0.07,12,0.9,0.966721,0.725333,0.241388
